# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
# NLP libraries
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#ML Libraries
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
# load data from database
def load_data():
    '''Function that loads that data from a SQL database and 
    returns the feature X and target variables Y'''    
    engine = create_engine('sqlite:///DisasterMessages.db')
    df = pd.read_sql("SELECT * FROM Messages",engine)
    #X = df.iloc[:,1]
    #Y = df.iloc[:,4:]
    #Only for testing
    X = df.iloc[:100,1]
    Y = df.iloc[:100,4:]
    return X,Y

#### Some testing to help define the function load data

In [9]:
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql("SELECT * FROM Messages",engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X0 = df.iloc[:10,1]
X0

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
6                       Storm at sacred heart of jesus
7    Please, we need tents and water. We are in Sil...
8      I would like to receive the messages, thank you
9    I am in Croix-des-Bouquets. We have health iss...
Name: message, dtype: object

In [ ]:
X1 = df.message.values
X1[:5]

In [14]:
df.iloc[:10,4:].head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    
    # Text Normalization(remove any characters that are not letters or numbers)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    tokens = word_tokenize(text)
    #remove stop words // Note: sometimes removing all the stopwords results in just one 
    #single word without a concrete meaning 
    
    #tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [17]:
#Checking the first two functions
X,y = load_data()
for message in X:
    tokens = tokenize(message)
    print(message)
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

Information about the National Palace-
['information', 'about', 'the', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### Additional References
* [Udacity Knowledge](https://knowledge.udacity.com/questions/52835)
* [Using GridSearch with MutiOutputClassification](https://knowledge.udacity.com/questions/45533)
* [Solving Multi-Label Classification Problems] (https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/)
* [sklearn.multiclass.OneVsRestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html)


In [18]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
    ])    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [19]:
#Loading data
X, y = load_data()
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train,y_train)
# predict on test data
y_pred = pipeline.predict(X_test)

C:\Users\Gustavo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Gustavo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Gustavo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Gustavo\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Gustavo\Anaconda3\lib\site-packages\sklearn\ensemble\forest

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [39]:
# Only works for single classification 
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    class_rep = classification_report(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()
    
    print("Category: ",y_test.name)
    print("Labels:", labels)
    print("Classification Report:\n", class_rep)
    print("Accuracy:", accuracy)

In [29]:
y0_test = y_test.iloc[:,0]

In [31]:
y0_pred = y_pred[:,0]

In [33]:
y0_test.name

'related'

In [42]:
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [44]:
for c in range(len(y_test.columns)):
    ycol_test=y_test.iloc[:,c]
    print(ycol_test.name)

related
request
offer
aid_related
medical_help
medical_products
search_and_rescue
security
military
child_alone
water
food
shelter
clothing
money
missing_people
refugees
death
other_aid
infrastructure_related
transport
buildings
electricity
tools
hospitals
shops
aid_centers
other_infrastructure
weather_related
floods
storm
fire
earthquake
cold
other_weather
direct_report


In [40]:
#Showing results
display_results(y0_test, y0_pred)

Category:  related
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.95      0.88      0.91        24

   micro avg       0.84      0.84      0.84        25
   macro avg       0.48      0.44      0.46        25
weighted avg       0.92      0.84      0.88        25

Accuracy: 0.84


In [57]:
#Multiclassification report 
def multiclass_report(y_test, y_pred):
    
    #iterating through the columns and calling sklearn's `classification_report` on each.
    for c in range(len(y_test.columns)):
        yc_test=y_test.iloc[:,c]
        yc_pred = y_pred[:,c]
           
        labels = np.unique(yc_pred)
        class_rep = classification_report(yc_test, yc_pred, labels=labels)
        accuracy = (yc_pred == yc_test).mean()
        print("----------------------------------------------------------------------")
        print("Category {}: {}".format(c+1,yc_test.name))
        print("Labels:", labels)
        print("Classification Report:\n", class_rep)
        print("Accuracy: {}\n".format(accuracy))
        print("----------------------------------------------------------------------")

In [58]:
#Showing Multioutput Classification Report
multiclass_report(y_test, y_pred)

----------------------------------------------------------------------
Category 1: related
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.95      0.88      0.91        24

   micro avg       0.84      0.84      0.84        25
   macro avg       0.48      0.44      0.46        25
weighted avg       0.92      0.84      0.88        25

Accuracy: 0.84

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 2: request
Labels: [0. 1.]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.50      0.91      0.65        11
         1.0       0.80      0.29      0.42        14

   micro avg       0.56      0.56      0.56        25
   macro avg       0.65      0.60      0.53        25
weighted avg       0.67      0.56      0.52        2

----------------------------------------------------------------------
Category 31: storm
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        25

   micro avg       1.00      1.00      1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

Accuracy: 1.0

----------------------------------------------------------------------
----------------------------------------------------------------------
Category 32: fire
Labels: [0.]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        25

   micro avg       1.00      1.00      1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

Accuracy: 1.0

----------------------------------------------------------------------
-----------------------------

In [ ]:
#Function for steps 3 to 5 
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # build pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])    
        
    # train classifier
    pipeline.fit(X_train,y_train)
    # predict on test data
    y_pred = pipeline.predict(X_test)
    # display results
    display_results(y_test, y_pred)

In [ ]:
#Running the model
main()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.